# Problem Statement
### To build a Logistic Regression model to predict whether a customer will churn or not based on customer data plans and its usage pattern. 


In [105]:
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# Import Data

In [2]:
Churn_Data=pd.read_excel('C:/Users/Mannu sharma/Desktop/BA/HGS/Data set -Churn modelling.xlsx',sheet_name='Data')

In [3]:
Churn_Data.head()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.7,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.7,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.0,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.0,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.0,3,166.7,113,41.0,7.42,10.1


In [4]:
Churn_Data_Description=pd.read_excel('C:/Users/Mannu sharma/Desktop/BA/HGS/Data set -Churn modelling.xlsx')
Churn_Data_Description

,Variables,Unnamed: 1
0,Churn,"1 if customer cancelled service, 0 if not"
1,AccountWeeks,number of weeks customer has had active account
2,ContractRenewal,"1 if customer recently renewed contract, 0 if not"
3,DataPlan,"1 if customer has data plan, 0 if not"
4,DataUsage,gigabytes of monthly data usage
5,CustServCalls,number of calls into customer service
6,DayMins,average daytime minutes per month
7,DayCalls,average number of daytime calls
8,MonthlyCharge,average monthly bill
9,OverageFee,largest overage fee in last 12 months


# EDA

In [9]:
pandas_profiling.ProfileReport(Churn_Data)

Number of variables,11
Number of observations,3333
Total Missing (%),0.0%
Total size in memory,286.5 KiB
Average record size in memory,88.0 B
Numeric,7
Categorical,0
Boolean,3
Date,0
Text (Unique),0
Rejected,1


### As we can see that Data Plan and Data usage are highly correlated (0.94598) and causing multi-colinearity so we can drop one variable "Data Usage"

In [4]:
Churn_Data.columns,Churn_Data.shape

(Index(['Churn', 'AccountWeeks', 'ContractRenewal', 'DataPlan', 'DataUsage',
        'CustServCalls', 'DayMins', 'DayCalls', 'MonthlyCharge', 'OverageFee',
        'RoamMins'],
       dtype='object'), (3333, 11))

In [5]:
Churn_Data=Churn_Data.drop(['DataUsage'],axis=1)

In [6]:
Churn_Data.columns,Churn_Data.shape

(Index(['Churn', 'AccountWeeks', 'ContractRenewal', 'DataPlan', 'CustServCalls',
        'DayMins', 'DayCalls', 'MonthlyCharge', 'OverageFee', 'RoamMins'],
       dtype='object'), (3333, 10))

In [7]:
Churn_Data.dtypes

Churn                int64
AccountWeeks         int64
ContractRenewal      int64
DataPlan             int64
CustServCalls        int64
DayMins            float64
DayCalls             int64
MonthlyCharge      float64
OverageFee         float64
RoamMins           float64
dtype: object

### As the data set is imbalance with 85.5% observations having Churn = 0 and 14.5% Churn = 1. Therefore, I have balanced the dataset by using SMOTE

In [8]:
from imblearn.over_sampling import SMOTE

In [9]:
sm = SMOTE(random_state= 2)

In [10]:
Response_var=Churn_Data['Churn']

In [11]:
Explanatory_var=Churn_Data.drop(['Churn'],axis=1)

### Splitting the dataset

In [106]:
from sklearn.model_selection import train_test_split

In [12]:
X_train,X_test,Y_train,Y_test=train_test_split(Explanatory_var,Response_var,test_size=0.3,random_state=123)

In [13]:
print("Number of Employees in X_train dataset: ", X_train.shape)
print("Number of Employees in y_train dataset: ", Y_train.shape)
print("Number of Employees in X_test dataset: ", X_test.shape)
print("Number of Employees in y_test dataset: ", Y_test.shape)

Number of Employees in X_train dataset:  (2333, 9)
Number of Employees in y_train dataset:  (2333,)
Number of Employees in X_test dataset:  (1000, 9)
Number of Employees in y_test dataset:  (1000,)


In [14]:
X_train_res, y_train_res = sm.fit_sample(X_train, Y_train.ravel())

In [15]:
print("Before OverSampling, counts of label '1': {}".format(sum(Y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(Y_train==0)))

Before OverSampling, counts of label '1': 349
Before OverSampling, counts of label '0': 1984 



In [16]:
print('After OverSampling, the shape of X_train: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of y_train: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

After OverSampling, the shape of X_train: (3968, 9)
After OverSampling, the shape of y_train: (3968,) 

After OverSampling, counts of label '1': 1984
After OverSampling, counts of label '0': 1984


In [17]:
y_train_res=pd.DataFrame(y_train_res)

In [18]:
y_train_res.head()

,0
0,0
1,0
2,0
3,0
4,0


In [19]:
X_train_res_df=pd.DataFrame(X_train_res,columns=['AccountWeeks', 'ContractRenewal', 'DataPlan', 'CustServCalls','DayMins', 'DayCalls', 'MonthlyCharge', 'OverageFee', 'RoamMins'])
X_train_res_df.head()


,AccountWeeks,ContractRenewal,DataPlan,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,70.0,1.0,0.0,1.0,213.4,86.0,54.0,10.24,5.7
1,67.0,1.0,0.0,2.0,109.1,134.0,31.0,7.12,10.9
2,122.0,1.0,1.0,2.0,146.4,104.0,75.1,4.49,15.6
3,107.0,0.0,0.0,1.0,234.1,91.0,56.9,8.16,10.0
4,105.0,1.0,0.0,2.0,125.4,116.0,46.8,13.08,11.4


In [20]:
data_cat=X_train_res_df[['ContractRenewal','DataPlan']]
data_num=X_train_res_df.drop(['ContractRenewal','DataPlan'],axis=1)
data_num_mm=X_train_res_df.drop(['ContractRenewal','DataPlan'],axis=1)

In [21]:
data_cat.head()

,ContractRenewal,DataPlan
0,1.0,0.0
1,1.0,0.0
2,1.0,1.0
3,0.0,0.0
4,1.0,0.0


In [22]:
data_num.head()

,AccountWeeks,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,70.0,1.0,213.4,86.0,54.0,10.24,5.7
1,67.0,2.0,109.1,134.0,31.0,7.12,10.9
2,122.0,2.0,146.4,104.0,75.1,4.49,15.6
3,107.0,1.0,234.1,91.0,56.9,8.16,10.0
4,105.0,2.0,125.4,116.0,46.8,13.08,11.4


# Standard Scaler

In [2]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [24]:
scaler=StandardScaler()

In [25]:
data_num=pd.DataFrame(scaler.fit_transform(data_num),columns=['AccountWeeks','CustServCalls','DayMins','DayCalls','MonthlyCharge','OverageFee','RoamMins'])

In [26]:
data_num.head()

,AccountWeeks,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,-0.821829,-0.565530,0.340533,-0.747182,-0.221450,-0.023729,-1.844886
1,-0.899447,0.113223,-1.328057,1.653581,-1.696634,-1.319999,0.179262
2,0.523546,0.113223,-0.731332,0.153104,1.131871,-2.412687,2.008781
3,0.135457,-0.565530,0.671692,-0.497103,-0.035448,-0.887909,-0.171071
4,0.083712,0.113223,-1.067290,0.753295,-0.683247,1.156208,0.373892


In [69]:
X_train_f=pd.merge(data_cat,data_num,on=X_train_res_df.index,how='inner')

In [86]:
X_train_f=X_train_f.drop(['key_0'],axis=1)

In [87]:
X_train_f.head()

,ContractRenewal,DataPlan,AccountWeeks,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,1.0,0.0,-0.821829,-0.565530,0.340533,-0.747182,-0.221450,-0.023729,-1.844886
1,1.0,0.0,-0.899447,0.113223,-1.328057,1.653581,-1.696634,-1.319999,0.179262
2,1.0,1.0,0.523546,0.113223,-0.731332,0.153104,1.131871,-2.412687,2.008781
3,0.0,0.0,0.135457,-0.565530,0.671692,-0.497103,-0.035448,-0.887909,-0.171071
4,1.0,0.0,0.083712,0.113223,-1.067290,0.753295,-0.683247,1.156208,0.373892


# Modelling (Logistic Regression)

* Applied Logistic Regression on the training set to train the model on scaled and unscaled data to get the best approach         possible.

* Used hyper parameters of L1 and L2 at different penalty rate to reduce overfitting.

* Tuned the model by cross validating with GridSearchCV to get the best combination of the hyper parameters possible with         scoring  ROC AUC.

* Evaluated the model on the test data with different evaluation metric like Confusion matrix, Accuracy ,Recall, Precision, ROC   AUC.


In [3]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression

In [88]:
log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000,10000]}
grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params)
grid_log_reg.fit(X_train_f, y_train_res)

# We automatically get the logistic regression with the best parameters.
log_reg = grid_log_reg.best_estimator_

C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sha

C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda

C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda

C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [89]:
log_reg_score = cross_val_score(log_reg, X_train_f, y_train_res,scoring='roc_auc',cv=5)

C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda

In [90]:
print('Logistic Regression Cross Validation Score: ', round(log_reg_score.mean() * 100, 2).astype(str) + '%')

Logistic Regression Cross Validation Score:  83.24%


In [91]:
data_cat_test=X_test[['ContractRenewal','DataPlan']]
data_num_test=X_test.drop(['ContractRenewal','DataPlan'],axis=1)

In [92]:
data_cat_test.head()

,ContractRenewal,DataPlan
465,0,0
2416,1,0
1785,1,1
1316,1,0
446,1,0


In [93]:
data_num_test=pd.DataFrame(scaler.transform(data_num_test),columns=['AccountWeeks','CustServCalls','DayMins','DayCalls','MonthlyCharge','OverageFee','RoamMins'])

C:\Users\Mannu sharma\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [94]:
data_num_test.head()

,AccountWeeks,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,-0.795957,0.791976,-0.222596,0.603247,-0.798696,-0.792351,1.230262
1,0.290693,0.113223,-0.577751,2.003693,-0.542142,1.056495,0.568522
2,-1.468644,0.113223,-0.900911,0.153104,0.375038,0.346040,-0.988516
3,-2.374185,0.113223,-0.145806,0.103088,-0.478004,0.184006,-0.677108
4,-0.356122,0.791976,-0.860916,0.753295,-0.862834,0.624405,-0.326775


In [95]:
data_num_test.isna().sum()

AccountWeeks     0
CustServCalls    0
DayMins          0
DayCalls         0
MonthlyCharge    0
OverageFee       0
RoamMins         0
dtype: int64

In [96]:
X_test_f1=pd.merge(data_cat_test,data_num_test,on=X_test.index,how='inner')

In [97]:
X_test_f1.head()

,key_0,ContractRenewal,DataPlan,AccountWeeks,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,465,0,0,-0.795957,0.791976,-0.222596,0.603247,-0.798696,-0.792351,1.230262
1,2416,1,0,0.290693,0.113223,-0.577751,2.003693,-0.542142,1.056495,0.568522
2,1785,1,1,-1.468644,0.113223,-0.900911,0.153104,0.375038,0.346040,-0.988516
3,1316,1,0,-2.374185,0.113223,-0.145806,0.103088,-0.478004,0.184006,-0.677108
4,446,1,0,-0.356122,0.791976,-0.860916,0.753295,-0.862834,0.624405,-0.326775


In [98]:
X_test_f1.isna().sum()

key_0              0
ContractRenewal    0
DataPlan           0
AccountWeeks       0
CustServCalls      0
DayMins            0
DayCalls           0
MonthlyCharge      0
OverageFee         0
RoamMins           0
dtype: int64

In [99]:
X_test_f1.shape

(1000, 10)

In [100]:
X_test_f1=X_test_f1.drop(['key_0'],axis=1)

In [101]:
X_test.shape

(1000, 9)

In [102]:
prediction=log_reg.predict(X_test_f1)

In [103]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,accuracy_score,f1_score,classification_report

In [104]:
print('Accuracy:', accuracy_score(Y_test, prediction))
print('F1 score:', f1_score(Y_test, prediction))
print('Recall:', recall_score(Y_test, prediction))
print('Precision:', precision_score(Y_test, prediction))
print('\n clasification report:\n', classification_report(Y_test,prediction))
print('\n confussion matrix:\n',confusion_matrix(Y_test, prediction))

Accuracy: 0.771
F1 score: 0.4735632183908047
Recall: 0.7686567164179104
Precision: 0.34219269102990035

 clasification report:
               precision    recall  f1-score   support

           0       0.96      0.77      0.85       866
           1       0.34      0.77      0.47       134

   micro avg       0.77      0.77      0.77      1000
   macro avg       0.65      0.77      0.66      1000
weighted avg       0.87      0.77      0.80      1000


 confussion matrix:
 [[668 198]
 [ 31 103]]


# Unscaled

In [47]:
log_reg_params1 = {"penalty": ['l1', 'l2'], 'C': [0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000,10000]}
grid_log_reg1 = GridSearchCV(LogisticRegression(), log_reg_params1)
grid_log_reg1.fit(X_train_res_df, y_train_res)

# We automatically get the logistic regression with the best parameters.
log_reg1 = grid_log_reg1.best_estimator_

C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sha

C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda

C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda

In [48]:
log_reg_score1 = cross_val_score(log_reg1, X_train_res_df, y_train_res,scoring='roc_auc',cv=5)

C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda

In [49]:
print('Logistic Regression Cross Validation Score: ', round(log_reg_score1.mean() * 100, 2).astype(str) + '%')

Logistic Regression Cross Validation Score:  83.24%


In [50]:
prediction1=log_reg1.predict(X_test)

In [51]:
print('Accuracy:', accuracy_score(Y_test, prediction1))
print('F1 score:', f1_score(Y_test, prediction1))
print('Recall:', recall_score(Y_test, prediction1))
print('Precision:', precision_score(Y_test, prediction1))
print('\n clasification report:\n', classification_report(Y_test,prediction1))
print('\n confussion matrix:\n',confusion_matrix(Y_test, prediction1))

Accuracy: 0.778
F1 score: 0.48611111111111105
Recall: 0.7835820895522388
Precision: 0.3523489932885906

 clasification report:
               precision    recall  f1-score   support

           0       0.96      0.78      0.86       866
           1       0.35      0.78      0.49       134

   micro avg       0.78      0.78      0.78      1000
   macro avg       0.66      0.78      0.67      1000
weighted avg       0.88      0.78      0.81      1000


 confussion matrix:
 [[673 193]
 [ 29 105]]


# MinMax

In [52]:
from sklearn.preprocessing import MinMaxScaler

In [53]:
minmax=MinMaxScaler()

In [54]:
data_num_mm=pd.DataFrame(minmax.fit_transform(data_num),columns=['AccountWeeks','CustServCalls','DayMins','DayCalls','MonthlyCharge','OverageFee','RoamMins'])

In [55]:
data_num_mm.head()

,AccountWeeks,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0.285124,0.111111,0.608324,0.521212,0.411100,0.578204,0.285
1,0.272727,0.222222,0.311003,0.812121,0.174717,0.402033,0.545
2,0.500000,0.222222,0.417332,0.630303,0.627955,0.253529,0.780
3,0.438017,0.111111,0.667332,0.551515,0.440904,0.460757,0.500
4,0.429752,0.222222,0.357469,0.703030,0.337102,0.738566,0.570


In [56]:
X_train_mmf=pd.concat([data_num_mm,data_cat],axis=1)

In [57]:
X_train_mmf.head()

,AccountWeeks,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins,ContractRenewal,DataPlan
0,0.285124,0.111111,0.608324,0.521212,0.411100,0.578204,0.285,1.0,0.0
1,0.272727,0.222222,0.311003,0.812121,0.174717,0.402033,0.545,1.0,0.0
2,0.500000,0.222222,0.417332,0.630303,0.627955,0.253529,0.780,1.0,1.0
3,0.438017,0.111111,0.667332,0.551515,0.440904,0.460757,0.500,0.0,0.0
4,0.429752,0.222222,0.357469,0.703030,0.337102,0.738566,0.570,1.0,0.0


In [58]:
log_reg_params2 = {"penalty": ['l1', 'l2'], 'C': [0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000,10000]}
grid_log_reg2 = GridSearchCV(LogisticRegression(), log_reg_params2)
grid_log_reg2.fit(X_train_mmf, y_train_res)

# We automatically get the logistic regression with the best parameters.
log_reg2 = grid_log_reg2.best_estimator_

C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sha

C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda

C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda

In [59]:
log_reg_score2 = cross_val_score(log_reg2, X_train_mmf, y_train_res,scoring='roc_auc',cv=5)

C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mannu sharma\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Mannu sharma\Anaconda

In [60]:
print('Logistic Regression Cross Validation Score: ', round(log_reg_score2.mean() * 100, 2).astype(str) + '%')

Logistic Regression Cross Validation Score:  81.29%


In [61]:
data_num_test_mm=X_test.drop(['ContractRenewal','DataPlan'],axis=1)

In [62]:
data_num_test_mm=pd.DataFrame(minmax.transform(data_num_test),columns=['AccountWeeks','CustServCalls','DayMins','DayCalls','MonthlyCharge','OverageFee','RoamMins'])

In [63]:
X_test_mmf1=pd.merge(data_cat_test,data_num_test_mm,on=X_test.index,how='inner')

In [64]:
X_test_mmf1.head()

,key_0,ContractRenewal,DataPlan,AccountWeeks,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,465,0,0,0.289256,0.333333,0.507982,0.684848,0.318602,0.473744,0.680
1,2416,1,0,0.462810,0.222222,0.444698,0.854545,0.359712,0.725014,0.595
2,1785,1,1,0.181818,0.222222,0.387115,0.630303,0.506680,0.628458,0.395
3,1316,1,0,0.037190,0.222222,0.521665,0.624242,0.369990,0.606437,0.435
4,446,1,0,0.359504,0.333333,0.394242,0.703030,0.308325,0.666290,0.480


In [65]:
X_test_mmf1=X_test_mmf1.drop(['key_0'],axis=1)

In [66]:
X_test_mmf1.head()

,ContractRenewal,DataPlan,AccountWeeks,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,0,0.289256,0.333333,0.507982,0.684848,0.318602,0.473744,0.680
1,1,0,0.462810,0.222222,0.444698,0.854545,0.359712,0.725014,0.595
2,1,1,0.181818,0.222222,0.387115,0.630303,0.506680,0.628458,0.395
3,1,0,0.037190,0.222222,0.521665,0.624242,0.369990,0.606437,0.435
4,1,0,0.359504,0.333333,0.394242,0.703030,0.308325,0.666290,0.480


In [67]:
prediction2=log_reg2.predict(X_test_mmf1)

In [68]:
print('Accuracy:', accuracy_score(Y_test, prediction2))
print('F1 score:', f1_score(Y_test, prediction2))
print('Recall:', recall_score(Y_test, prediction2))
print('Precision:', precision_score(Y_test, prediction2))
print('\n clasification report:\n', classification_report(Y_test,prediction2))
print('\n confussion matrix:\n',confusion_matrix(Y_test, prediction2))

Accuracy: 0.462
F1 score: 0.13782051282051283
Recall: 0.3208955223880597
Precision: 0.08775510204081632

 clasification report:
               precision    recall  f1-score   support

           0       0.82      0.48      0.61       866
           1       0.09      0.32      0.14       134

   micro avg       0.46      0.46      0.46      1000
   macro avg       0.45      0.40      0.37      1000
weighted avg       0.72      0.46      0.55      1000


 confussion matrix:
 [[419 447]
 [ 91  43]]


## Evaluation scores

# Business Interpretation

* Recall should be the most important metric as we can find, out of actual churners how many are we able to predict correctly,     as we can’t bear predicting churners as non churners and lose customers. Therefore the model with better recall will improve   the business.

* Accuracy is second important metric as we can identify, out of all customers what percentage we have predicted churners and     non-churners correctly

* Precision will come next in this scenario. We can bear slightly low precision value because predicting non churners as           churners can cause less harm to the business. 
